In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install xlrd
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sla = pd.read_excel("../input/shopee-code-league-20/_DA_Logistics/SLA_matrix.xlsx")
deliver = pd.read_csv("../input/shopee-code-league-20/_DA_Logistics/delivery_orders_march.csv")

**Cleaning data**

In [ ]:
# Fill null data
sla = sla.fillna(-1)
deliver = deliver.fillna(-1)

In [ ]:
sla.dtypes

In [ ]:
deliver.dtypes

In [ ]:
sla.head()

In [ ]:
deliver.head()

In [ ]:
# Convert datetime
deliver['1st_deliver_attempt'] = pd.to_datetime(deliver['1st_deliver_attempt'], unit = 's').dt.strftime('%Y-%m-%d')
# deliver['1st_deliver_attempt'] = deliver['1st_deliver_attempt'].dt.round('1s')

In [ ]:
# Convert datetime
deliver['2nd_deliver_attempt'] = pd.to_datetime(deliver['2nd_deliver_attempt'], unit = 's').dt.strftime('%Y-%m-%d')
# deliver['2nd_deliver_attempt'] = deliver['2nd_deliver_attempt'].dt.round('1s')

In [ ]:
# Convert datetime
deliver['pick'] = pd.to_datetime(deliver['pick'], unit='s').dt.strftime('%Y-%m-%d')

In [ ]:
# Convert upper case to lower case
deliver['buyeraddress'] = deliver['buyeraddress'].str.lower()
deliver['selleraddress']  = deliver['selleraddress'].str.lower()

In [ ]:
deliver.head()

In [ ]:
days = ["Mon", "Tue"," Wed"," Thur", "Fri", "Sat", "Sun"]
public_hol = ['2020-03-08','2020-03-25','2020-03-30','2020-03-31']

In [ ]:
locations = ["metro manila","luzon","visayas","mindanao"]
deliver.loc[deliver['buyeraddress'].str.contains('metro manila'), 'buyeraddress'] = 'metro manila'
deliver.loc[deliver['buyeraddress'].str.contains('luzon'), 'buyeraddress'] = 'luzon'
deliver.loc[deliver['buyeraddress'].str.contains('visayas'), 'buyeraddress'] = 'visayas'
deliver.loc[deliver['buyeraddress'].str.contains('mindanao'), 'buyeraddress'] = 'mindanao'
deliver.head()

In [ ]:
locations = ["metro manila","luzon","visayas","mindanao"]
deliver.loc[deliver['selleraddress'].str.contains('metro manila'), 'selleraddress'] = 'metro manila'
deliver.loc[deliver['selleraddress'].str.contains('luzon'), 'selleraddress'] = 'luzon'
deliver.loc[deliver['selleraddress'].str.contains('visayas'), 'selleraddress'] = 'visayas'
deliver.loc[deliver['selleraddress'].str.contains('mindanao'), 'selleraddress'] = 'mindanao'
deliver.head()

In [ ]:
def working_days(df):
    if df['buyeraddress'] == "metro manila" and df['selleraddress'] == 'metro manila':
        return 3
    elif (df['buyeraddress'] == "metro manila" or df['buyeraddress'] == 'luzon') and (df['selleraddress'] == 'metro manila' or  df['selleraddress'] == 'luzon'):
        return 5
    else:
        return 7
        

In [ ]:
deliver['working_days'] = deliver.apply(lambda row: working_days(row), axis=1)

In [ ]:
deliver.head(10)

In [ ]:
holidays = ["2020-03-08", "2020-03-25", "2020-03-30", "2020-03-31"]
deliver["first_try"] = np.busday_count(deliver["pick"], deliver["1st_deliver_attempt"], weekmask="1111110", holidays = holidays)
deliver["second_try"] = np.busday_count(deliver["1st_deliver_attempt"], deliver["2nd_deliver_attempt"], weekmask="1111110", holidays = holidays) 

In [ ]:
deliver.head()

In [ ]:
def isLate(df):
    if df['first_try'] > df['working_days'] or (df['second_try'] > 3):
        return 1
    else:
        return 0

In [ ]:
deliver['isLate'] = deliver.apply(lambda row: isLate(row), axis=1)

In [ ]:
deliver.head()

In [ ]:
result = deliver[['orderid','isLate']].copy()
result.head()
print(len(result.loc[result['isLate'] == 1]))